### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 we can possibly look for jump operator of the form:

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or

$
A = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [2]:
@polyvar e[1]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [3]:
method = "fix-Phi-up_"

@polyvar a[1:4]
@polyvar b[1:4]


#"just-zero-down_ "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2] + im*b[2] 
#          0                -a[1]-im*b[1]  ]


#"fix-Phi-up_: "

Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
          a[3] + im*b[3]   -a[1]-im*b[1] ]


#"zero Temperature and fix-Phi-up_: "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
#          0                -a[1]-im*b[1] ]

#"zero Temperature and fix-Phi-diag_: "

#Aˢʸᵐᵇ = [ a[1]     a[2]+im*b[2] 
#          0        -a[1]        ]




2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂
 a₃ + (0+1im)b₃  -a₁ + (0-1im)b₁

In [4]:
pwd()

"/home/zah/POP23"

In [5]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

test_files = ["State_D"*string(n) for n=1:20];

/home/zah/POP23/DATA/


### Train models loop

In [6]:
ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
t = convert(Vector{Float64}, tᵍ)
ρ = convert(Vector{Matrix{ComplexF64}}, ρᵍ)
LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])

γᶠ = parse(Float64, "0.079477")


0.079477

In [7]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP_fit_trainB4_"*method*date_and_time_string * ".h5"
tests_data_file_name = "POP_fit_testD20_"*method*date_and_time_string * ".h5"

println(models_file)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    println("gamma = ", γᵢ)

    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    
    objₑₓ = 0
    for (ρ,t) in zip([ρᵍ, ρᵉ, ρˣ, ρʸ], [tᵍ, tᵉ, tˣ, tʸ])
        
        t = convert(Vector{Float64}, t)
        
        # Convert cut ρ series:
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
        objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
        
    end # of files (initial states) loop
    
    
    Hᴸᴹᴱ = [ 25.126        0
             0             0    ]
    γᶠ = parse(Float64, γᵢ)

    Aᴸᴹᴱ = [ 0    √(γᶠ)
             0     0. + 0im  ]

    ρᵍₗₘₑ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], convert(Vector{Float64}, tᵍ), Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρᵉₗₘₑ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], convert(Vector{Float64}, tᵉ), Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρˣₗₘₑ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], convert(Vector{Float64}, tˣ), Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρʸₗₘₑ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], convert(Vector{Float64}, tʸ), Hᴸᴹᴱ, [Aᴸᴹᴱ])
    
    
    objₗₘₑ = 0
    for (ρ,t) in zip([ρᵍₗₘₑ, ρᵉₗₘₑ, ρˣₗₘₑ, ρʸₗₘₑ], [tᵍ, tᵉ, tˣ, tʸ])
        
        t = convert(Vector{Float64}, t)

        # Convert cut ρ series:
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
        objₗₘₑ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
        
    end # of files (initial states) loop
    
    
    println(" POP on ansatz LME simulation")
    
    solˢⁱᵈₗₘₑ, best_methodₐₙ = LiPoSID.sos_min_newton(objₗₘₑ) 
    
    println("Best metod for POP on LME: ", best_methodₐₙ)
    
    solₗₘₑ = [e[1], h[1], h[2], a[1], a[2], a[3], b[1], b[3]] => [25.126, 0, 0, 0,  √(γᶠ), 0, 0, 0]
    
    obj_val_lme = convert(Float64, subs(objₗₘₑ, solₗₘₑ))
    
    obj_val_sid_on_lme = convert(Float64, subs(objₗₘₑ, solˢⁱᵈₗₘₑ))
    
    if obj_val_sid_on_lme < obj_val_lme
        println(" simulated LME overfitting")
    end
    
    Hˢⁱᵈₗₘₑ = subs(Hˢʸᵐᵇ, solˢⁱᵈₗₘₑ)
    Aˢⁱᵈₗₘₑ = subs(Aˢʸᵐᵇ, solˢⁱᵈₗₘₑ)
      
    println(" POP on ansatz EXACT Kurt data ")
    
    solₑₓ, best_methodₑₓ = LiPoSID.sos_min_newton(objₑₓ) 
    
    println("Best metod for POP on EXACT: ", best_methodₑₓ)
    
    obj_val_lme_on_exact = convert(Float64, subs(objₑₓ, solₗₘₑ))
    
    obj_val_sid_on_exact = convert(Float64, subs(objₑₓ, solₑₓ))
    
    if obj_val_sid_on_exact < obj_val_lme_on_exact
        println(" Exact data overfitting")
    end
    

    Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓ)
    Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓ)
    
    H = DenseOperator(basis, Hˢⁱᵈₑₓ)
    A = DenseOperator(basis, Aˢⁱᵈₑₓ)

    
    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        
        
        γ_group["obj_val_lme"] = obj_val_lme
        γ_group["obj_val_sid_on_lme"] = obj_val_sid_on_lme
        
        γ_group["obj_val_lme_on_exact"] = obj_val_lme_on_exact
        γ_group["obj_val_sid_on_exact"] = obj_val_sid_on_exact

        γ_group["H_sid_exact"] = convert.(ComplexF64, Hˢⁱᵈₑₓ)
        γ_group["A_sid_exact"] = convert.(ComplexF64, Aˢⁱᵈₑₓ)
        
        γ_group["H_lme"] = convert.(ComplexF64, Hᴸᴹᴱ)
        γ_group["A_lme"] = convert.(ComplexF64, Aᴸᴹᴱ)
        
        γ_group["H_sid_lme"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑ)
        γ_group["A_sid_lme"] = convert.(ComplexF64, Aˢⁱᵈₗₘₑ)

        
    end # of HDF5 writing
    
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        t = convert(Vector{Float64}, tₛ)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(t, ρᵗˢᵗ[1], DenseOperator(basis,Hᴸᴹᴱ), [DenseOperator(basis,Aᴸᴹᴱ)])
        tˢⁱᵈₗₘ, ρˢⁱᵈₗₘₑ  = timeevolution.master(t, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑ), [DenseOperator(basis,Aˢⁱᵈₗₘₑ)])
        tˢⁱᵈₑₓ, ρˢⁱᵈₑₓ  = timeevolution.master(t, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₑₓ), [DenseOperator(basis,Aˢⁱᵈₑₓ)])   
        

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴸᴹᴱ)]
        Fˢⁱᵈᴸᴹᴱₗₘₑ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₗₘₑ, ρᴸᴹᴱ)]
        Fˢⁱᵈᵉˣₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈₑₓ)]
                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_lme_exact"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["F_sidlme_lme"] = convert.(Float64, Fˢⁱᵈᴸᴹᴱₗₘₑ)
            init_state_group["F_sidexact_exact"] = convert.(Float64, Fˢⁱᵈᵉˣₑₓ)
            
        end
                    
        print(minimum(Fᴸᴹᴱₑₓ), minimum(Fˢⁱᵈᴸᴹᴱₗₘₑ), minimum(Fˢⁱᵈᵉˣₑₓ))
        println(" ")
    
    end
                
    println()
    println()
                            

end

POP_fit_trainB4_fix-Phi-up_2023-Aug-01_at_20-42.h5
gamma = 0.079477
 POP on ansatz LME simulation
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 1.7332037275317426e-8

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a locally optimal solution by Ipopt, giving an upper bound: 0.01054168.
The relative optimality gap is: 1.054166%.
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 2.2624657123

State_D10 0.99899482220359910.99999999999972820.9962017837209791 
State_D11 0.99882898529036370.9999999999997770.996071437584966 
State_D12 0.99883370462813540.9999999999997780.99607771177828 
State_D13 0.99809719710336380.99999999999993520.9958193264761114 
State_D14 0.9981294057959960.99999999999993560.9958250576594643 
State_D15 0.9987200680942190.99999999999980790.9960042528498934 
State_D16 0.99872051243386570.99999999999980760.9960041305751403 
State_D17 0.99850183419307970.99999999999985590.995913150760112 
State_D18 0.99851380960311230.9999999999998570.9959175065832938 
State_D19 0.99899505916456450.99999999999972820.9962013147682712 
State_D20 0.99789912772182470.99999999999998620.9958040969593059 


gamma = 2.5133
 POP on ansatz LME simulation
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 1.5879551564235625e-9
Global optimality certified with relative optimality ga

State_D8 0.91313353040438890.99999999999999790.9872449828480925 
State_D9 0.91105351726581320.99999999999999830.9996045152390973 
State_D10 0.91609992316778430.99999999999999840.9996393856066992 
State_D11 0.91487693128799170.99999999999999710.9791308002004737 
State_D12 0.93620316440096230.9999999999999980.9960928555989563 
State_D13 0.91124993894953730.99999999999999720.9781511726417409 
State_D14 0.9119663713146580.9999999999999980.995192531334133 
State_D15 0.93089575256946620.99999999999999790.9888831793437165 
State_D16 0.93089575257037750.99999999999999790.9987132426940916 
State_D17 0.91883893874296220.99999999999999660.9731104302013389 
State_D18 0.91313353040335010.99999999999999780.9976315798219683 
State_D19 0.91609992316931390.99999999999999830.9946794369191561 
State_D20 0.91105351726791620.99999999999999810.9936644092998217 


gamma = 79.477
 POP on ansatz LME simulation
*********************************** TSSOS ***********************************
Version 1.0.0, develope

In [8]:
tests_data_file_name

"POP_fit_testD20_fix-Phi-up_2023-Aug-01_at_20-42.h5"